In [1]:
import sys
import os
sys.path.append('../')
%load_ext autoreload
%autoreload 2

import dice_ml
import torch
from models.run_MLP import MLP
from utils.parser import *
from models.data_process import *
from sklearn.model_selection import train_test_split

args = parse_args()
device = args.device

In [20]:
datafile='../'+args.data_path + '/final_data.csv'
adult_data = pd.read_csv(datafile, header = 0)
adult_data.drop('id',axis=1,inplace = True)
target = adult_data['income']
# adult_data.drop('income',axis=1,inplace = True)

train_dataset, test_dataset, y_train, y_test = train_test_split(adult_data,
                                                                target,
                                                                test_size=0.2,
                                                                random_state=0,
                                                                stratify=target)
print(train_dataset.head())
x_train = train_dataset.drop('income', axis=1)
x_test = test_dataset.drop('income', axis=1)

       age workclass     education  educational-num      marital-status  \
29101   19   Private  Some-college               10       Never-married   
10606   50   Private  Some-college               10  Married-civ-spouse   
9414    62   Private  Some-college               10  Married-civ-spouse   
1747    20   Private       HS-grad                9       Never-married   
10677   25   Private     Bachelors               13       Never-married   

              occupation    relationship   race  gender  hours-per-week  \
29101     Prof-specialty  Other-relative  White  Female              30   
10606  Machine-op-inspct         Husband  White    Male              40   
9414     Exec-managerial         Husband  White    Male              40   
1747        Adm-clerical   Not-in-family  White  Female              40   
10677       Adm-clerical       Own-child  White  Female              40   

      native-country  income  
29101  United-States       0  
10606  United-States       0  
9414 

In [17]:
d = dice_ml.Data(dataframe=train_dataset, continuous_features=['age', 'educational-num','hours-per-week'], outcome_name='income')

In [18]:
model = MLP().to(device)
ckpt = torch.load('../'+args.model_path+f'MPL_{args.epoch}.pth', map_location='cpu')
model.load_state_dict(ckpt)
model.eval()


m = dice_ml.Model(model=model, backend='PYT',  func="ohe-min-max")
exp = dice_ml.Dice(d, m, method="gradient")

/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)


In [21]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(x_test[1:3], total_CFs=4, desired_class="opposite")
# highlight only the changes
dice_exp.visualize_as_dataframe(show_only_changes=True)

  0%|          | 0/2 [00:00<?, ?it/s]/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  retur

No Counterfactuals found for the given configuation,  perhaps try with different values of proximity (or diversity) weights or learning rate... ; total time taken: 00 min 30 sec


/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Des

Diverse Counterfactuals found! total time taken: 00 min 04 sec
Query instance (original outcome : 0)


,age,workclass,education,educational-num,marital-status,occupation,relationship,race,gender,hours-per-week,native-country,income
0,46.0,Private,10th,6.0,Married-spouse-absent,Other-service,Not-in-family,Asian-Pac-Islander,Male,37.0,China,0.0



No counterfactuals found!
Query instance (original outcome : 0)


,age,workclass,education,educational-num,marital-status,occupation,relationship,race,gender,hours-per-week,native-country,income
0,47.0,Private,Some-college,10.0,Divorced,Exec-managerial,Not-in-family,White,Female,48.0,United-States,0.0



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,educational-num,marital-status,occupation,relationship,race,gender,hours-per-week,native-country,income
0,-,-,-,-,Married-AF-spouse,-,Husband,Black,-,-,Hungary,1
1,90.0,Federal-gov,Doctorate,16.0,Married-AF-spouse,-,Wife,-,Male,99.0,Italy,1
2,69.0,-,Doctorate,15.0,Married-civ-spouse,-,Wife,-,-,-,-,1
3,90.0,-,Doctorate,-,Married-civ-spouse,-,Wife,Black,Male,-,-,1


In [22]:
imp = exp.local_feature_importance(x_test[0:1], total_CFs=10)
print(imp.local_importance)

  0%|          | 0/1 [00:00<?, ?it/s]/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  retur

Diverse Counterfactuals found! total time taken: 00 min 18 sec
[{'education': 0.9, 'age': 0.8, 'workclass': 0.7, 'hours-per-week': 0.7, 'occupation': 0.6, 'educational-num': 0.6, 'race': 0.5, 'native-country': 0.4, 'relationship': 0.3, 'marital-status': 0.2, 'gender': 0.1}]


In [30]:
imp = exp.global_feature_importance(x_test[0:10])
print(imp.summary_importance)

  0%|          | 0/10 [00:00<?, ?it/s]/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  retu

Diverse Counterfactuals found! total time taken: 00 min 19 sec


/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Des

No Counterfactuals found for the given configuation,  perhaps try with different values of proximity (or diversity) weights or learning rate... ; total time taken: 01 min 20 sec


/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Des

...

Diverse Counterfactuals found! total time taken: 00 min 17 sec


/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Des

Diverse Counterfactuals found! total time taken: 00 min 16 sec


/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Desktop/graduate-project/explanation_model/counterfactual/../models/run_MLP.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(out)
/Users/heavenmei/Des

Diverse Counterfactuals found! total time taken: 00 min 18 sec
{'education': 0.8533333333333334, 'age': 0.7333333333333333, 'relationship': 0.68, 'hours-per-week': 0.68, 'workclass': 0.6533333333333333, 'educational-num': 0.6133333333333333, 'occupation': 0.5466666666666666, 'marital-status': 0.49333333333333335, 'native-country': 0.4666666666666667, 'race': 0.44, 'gender': 0.21333333333333335}
